In [2]:
# %pip install git+https://github.com/samoturk/mol2vec

In [4]:
%pip install gensim

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


[Ссылка]('https://sci-hub.ru/https://doi.org/10.1021/acs.jcim.7b00616') на статью про mol2vec

In [5]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit import RDLogger

import pandas as pd
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
RDLogger.DisableLog('rdApp.warning')

In [8]:
from rdkit import Chem
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
import requests


# import numpy/pandas ---------------------------------------------------------
import numpy as np
import pandas as pd

## Реализуем функционал гененерации эмбэдингов с помощью mol2vec

In [9]:
url = 'https://github.com/samoturk/mol2vec_notebooks/raw/master/Notebooks/model_300dim.pkl'
file_to_download = requests.get(url, allow_redirects=True)

open('model_300dim.pkl', 'wb').write(file_to_download.content)
w2v_model = word2vec.Word2Vec.load('model_300dim.pkl')

In [17]:
def smiles2vec(smiles):
    
    smiles = list(smiles)
    mol = [Chem.MolFromSmiles(x) for x in tqdm(smiles)]
    sentence = [MolSentence(mol2alt_sentence(x, radius=1)) for x in tqdm(mol)]
    
    def sentences2vec(sentences, model, unseen=None):
        """Generate vectors for each sentence (list) in a list of sentences. Vector is simply a
        sum of vectors for individual words.

        Parameters
        ----------
        sentences : list, array
            List with sentences
        model : word2vec.Word2Vec
            Gensim word2vec model
        unseen : None, str
            Keyword for unseen words. If None, those words are skipped.
            https://stats.stackexchange.com/questions/163005/how-to-set-the-dictionary-for-text-analysis-using-neural-networks/163032#163032

        Returns
        -------
        np.array
        """
        keys = set(model.wv.key_to_index.keys())
        vec = []
        if unseen:
            unseen_vec = model.wv.word_vec(unseen)

        for sentence in tqdm(sentences):
            if unseen:
                vec.append(sum([model.wv.word_vec(y) if y in set(sentence) & keys
                           else unseen_vec for y in sentence]))
            else:
                vec.append(sum([model.wv.word_vec(y) for y in sentence
                                if y in set(sentence) & keys]))
        return np.array(vec)
    
    
    embeddings = np.array([vec for vec in sentences2vec(sentence, w2v_model, unseen='UNK')])
    
    return pd.DataFrame(embeddings)


def main():
    exsample_smiles = ['C1CCCCC1', 'c1ccccc1']

    print(smiles2vec(exsample_smiles))


if __name__ == '__main__':
    main()  
    

100%|██████████| 2/2 [00:00<00:00, 8914.57it/s]

         0         1         2   ...        97        98        99
0  1.727892  1.026292  1.506369  ...  2.622092 -4.635670  1.371937
1 -0.789778  0.448608 -1.514847  ...  4.251935 -2.027579 -1.113355

[2 rows x 100 columns]


## Подгружаем данные и считаем

In [18]:
df = pd.read_table('/home/jupyter/datasphere/project/Расчёт дискрипторов/mapper_moleculares.tsv')
df.head()

,mol,prepare_mol
0,O=[N+:22]([O-])[C:25]1=[C:26]([Cl:34])[C:27]([...,O=[N+]([O-])[C]1=[C]([Cl])[C]([Cl])=N[C]([Cl])...
1,C[Si-](C)(C)(F)F,C[Si-](C)(C)(F)F
2,OC(=O)C1=CN2C=C(C(F)(F)F)[CH:19]=[C:14](Cl)[C:...,OC(=O)C1=CN2C=C(C(F)(F)F)[CH]=[C](Cl)[C]2=N1
3,CC1(C)[N:19]([C:24]([CH:25]([F:26])[F:27])=[O:...,CC1(C)[N]([C]([CH]([F])[F])=[O])[C@H]([CH2][F]...
4,[CH2:1]([CH2:2][CH2:3][CH2:4][CH2:5][CH3:6])[C...,[CH2]([CH2][CH2][CH2][CH2][CH3])[C]1=[C]([CH]=...


In [19]:
smileses = df.prepare_mol.unique()
res = smiles2vec(smileses)
res['smiles'] = list(smileses)
res.head()

100%|██████████| 621478/621478 [03:28<00:00, 2974.27it/s]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,smiles
0,-1.497707,4.112063,0.450655,-0.079097,-1.512574,-0.733806,-2.875139,-1.173212,0.718233,-10.539660,10.318323,-5.703891,-3.169460,0.383737,-2.358247,1.095210,-2.004729,2.679191,-2.937520,-3.498560,-6.457624,3.398917,-0.535400,5.615125,-0.660811,-2.815016,0.463224,-1.191941,-2.620518,1.445254,-0.931193,4.762228,12.033413,10.116756,-3.579432,-4.736619,6.317016,5.409988,-4.388782,11.674562,...,-0.212960,3.657190,-2.477092,-2.665408,-4.027892,0.814873,8.758249,-2.101277,4.285078,-3.158729,-4.997027,11.285654,1.604119,-7.590145,-3.895738,2.613756,5.042601,2.512323,-8.856397,6.103949,7.326143,0.761880,-5.456260,-7.971244,8.147049,3.912544,10.392677,-2.003241,-3.441931,9.331133,3.457170,-9.134859,-4.142196,1.647332,-12.420591,1.399055,10.382340,-3.711495,0.558785,O=[N+]([O-])[C]1=[C]([Cl])[C]([Cl])=N[C]([Cl])...
1,-0.744123,0.685963,-2.325320,0.949545,-1.152043,-2.456861,1.341186,-1.012225,-1.841382,-4.244472,4.454873,-6.527304,1.920393,-3.424803,-2.395674,2.507599,-1.752363,2.547493,-1.467167,-1.238578,-5.396310,2.564380,-1.195529,-3.153712,3.106251,2.150328,0.566598,0.196749,5.610125,-0.370715,2.591770,-1.265008,4.155287,3.499548,-1.674896,-1.732210,-0.372957,0.959619,-1.782793,3.590945,...,0.279060,0.970818,1.310265,-5.962324,-0.123200,-0.748672,3.012232,-4.491597,2.483390,-2.169155,-1.253826,3.651784,1.611469,0.171115,0.999543,3.228842,0.399839,-3.957694,-1.146780,3.087837,-0.024685,-1.559541,0.038660,-1.132573,1.950820,-1.035421,3.686740,-1.480967,-2.415655,6.346478,0.483593,-4.744343,-4.122983,0.319699,-3.990093,2.112992,2.554560,-1.306344,-1.185148,C[Si-](C)(C)(F)F
2,-1.834927,1.249379,-3.459645,2.268413,-5.094517,-2.831186,-5.581830,-5.429892,-0.480470,-10.738225,13.608158,-5.331883,1.090693,-4.487108,-7.194826,-3.594958,-4.914903,2.069929,-5.298110,-1.419245,-9.128433,5.567597,-12.225845,3.771619,2.386173,-5.367375,-4.430055,-2.432287,3.621405,-0.272935,2.357718,4.022657,14.294954,6.820892,-1.967933,-9.711524,6.684253,1.967004,-8.681560,14.634129,...,3.945039,2.533989,-2.136686,-3.793245,-3.576746,-2.329648,8.146861,-0.935044,14.093030,-4.149989,-9.501831,9.249758,0.937556,-6.026664,-4.779413,-0.496039,3.833607,1.063350,-10.969135,11.993348,5.504348,1.772928,-6.237563,-15.319273,12.180788,1.691801,17.352947,-4.469793,0.775371,12.040432,1.559295,-6.337942,-8.700478,3.363512,-18.810366,-2.426791,10.525679,-6.218207,-1.885402,OC(=O)C1=CN2C=C(C(F)(F)F)[CH]=[C](Cl)[C]2=N1
3,-3.108778,2.189721,-6.981885,-0.308557,-10.750094,-8.928565,-10.185752,-12.779259,-3.497235,-11.831833,15.344705,-15.833057,5.074119,-5.095232,-13.127588,-6.270169,-6.413956,3.659870,-0.824117,-12.003437,-12.388411,10.049482,-18.372272,3.426017,3.292769,-4.166801,-7.920877,-4.599417,11.714268,-5.331777,-1.713330,7.732607,18.125441,3.014438,-15.675670,-13.676994,12.859793,8.866338,-10.882317,18.233232,...,-4.566869,4.462863,-1.029608,-6.701056,-13.467010,-4.460234,16.857643,3.939813,15.486722,-5.225771,-10.637444,14.116968,-6.819273,-7.685589,-5.608485,-0.221647,2.666635,-7.386281,-11.298620,15.352395,7.304541,1.428864,-7.223143,-26.609324,16.006460,5.236518,31.756235,-5.982730,-1.725215,18.752913,5.350545,-6.310513,-17.930960,16.401827,-27.253578,3.307081,16.941963,-14.769436,-4.671370,CC1(C)[N]([C]([CH]([F])[F])=[O])[C@H]([CH2][F]...
4,4.836612,2.788502,-7.164144,-0.802584,-7.463440,-9.527348,-10.311643,-17.341757,-13.010865,-15.522384,19.164484,-15.636586,9.540207,-3.349625,-12.806125,-5.443763,2.927195,14.104742,-2.976399,-7.724328,-7.708750,11.105650,-10.319038,13.207362,6.696754,-9.778647,-1.884082,-3.784556,3.524303,-2.679704,1.821564,14.452200,20.307026,4.817284,-17.450634,-15.806260,17.837591,14.470726,-13.144598,16.338905,...,-0.305673,19.215727,1.045292,-3.371313,-10.567642,0.395174,29.251671,-2.448852,13.422495,-3.

In [20]:
res.to_pickle('mol2vec_feature.pickle')